# Защита персональных данных клиентов страховой компании

**Описание проекта**

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

План работы:

1. Загрузим и изучим данные.
2. Умножим признаки на обратимую матрицу. Проверим изменится ли качество линейной регрессии?
   1. Изменится. Приведите примеры матриц.
   2. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
3. Предложим алгоритм преобразования данных для решения задачи. Покажем, почему качество линейной регрессии не поменяется.
4. Запрограммируем этот алгоритм, применив матричные операции. Проверим, что качество линейной регрессии из `sklearn` не отличается до и после преобразования с помощью метрики `R2`.

**Описание данных**

Набор данных находится в файле `/datasets/insurance.csv`.

1. **Признаки:** пол, возраст и зарплата застрахованного, количество членов его семьи.
2. **Целевой признак:** количество страховых выплат клиенту за последние 5 лет.

In [25]:
import pandas as pd
import numpy as np
import plotly.express as px
import optuna

from collections import defaultdict
from IPython.display import display

from fast_ml import eda
from ydata_profiling import ProfileReport

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor

from sklearn.metrics import r2_score

In [10]:
FIG_WIDTH = 9 * 100
FIG_HEIGHT = 5 * 100
RANDOM_SEED = 42

In [11]:
try:
    raw_claims = pd.read_csv('insurance.csv')
except:
    raw_claims = pd.read_csv('/datasets/insurance.csv')

## Исследовательский анализ данных

### Описание данных

Изучим основные зависимости в данных перед тем, как мы будем использовать их в алгоритмах машинного обучения.

Таблицы-резюме:

In [12]:
display(eda.df_info(raw_claims))

,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
Пол,int64,Numerical,2,"[1, 0]",0,0.0
Возраст,float64,Numerical,46,"[41.0, 46.0, 29.0, 21.0, 28.0, 43.0, 39.0, 25....",0,0.0
Зарплата,float64,Numerical,524,"[49600.0, 38000.0, 21000.0, 41700.0, 26100.0, ...",0,0.0
Члены семьи,int64,Numerical,7,"[1, 0, 2, 4, 3, 5, 6]",0,0.0
Страховые выплаты,int64,Numerical,6,"[0, 1, 2, 3, 5, 4]",0,0.0


In [13]:
display(round(raw_claims.describe(), 2))

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.0,5000.00,5000.00,5000.00,5000.00
mean,0.5,30.95,39916.36,1.19,0.15
std,0.5,8.44,9900.08,1.09,0.46
min,0.0,18.00,5300.00,0.00,0.00
25%,0.0,24.00,33300.00,0.00,0.00
50%,0.0,30.00,40200.00,1.00,0.00
75%,1.0,37.00,46600.00,2.00,0.00
max,1.0,65.00,79000.00,6.00,5.00


Детальный обзор:

In [14]:
ProfileReport(raw_claims).to_widgets()

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Причешем немного датасет перед анализом:

In [17]:
df_claims = (
    raw_claims
    .copy()
    .rename(columns={
        'Пол': 'is_male', 'Возраст': 'age', 'Зарплата': 'income',
        'Члены семьи': 'family_members_count', 'Страховые выплаты': 'claims_count'
    })
    .astype({
        'age': 'int64', 'income': 'int64'
    })
)

Предварительные наблюдения:

1. В датасете нет пропусков, но есть повторяющиеся строки. Они не должны повлиять на процесс решения бизнес-задачи, поэтому оставим их как есть.

2. Датасет является сбалансированным по признаку `is_male`, с равным представлением между двумя полами. Это хорошо поскольку снижает риск смещения в данных.

3. В наборе данных представлен широкий диапазон возрастов, от 18 до 65 лет со средним значением около 31 года. Это означает, что данные покрывают широкую группу клиентов.

4. Для колонки `claims_count` мы можем заметить, что среднее значение составляет `0.15`, что ближе к минимальному значению `0`, a медиана также равна `0`. Это говорит о том, что большое количество людей в наборе данных не получают страховые выплаты.

## Преобразование признаков

Проверим, какие преобразования исходных мы можем выполнить, чтобы качество ML моделей не ухудшилось.

### Умножение на обратимую матрицу

Сначала, посмотрим, что будет если признаки умножить на обратимую матрицу. Эту задачу можно решить в явном виде. Для этого, допустим, что:

1. $X$ - матрица признаков
2. $y$ - вектор целевого признака
3. $w$ - матрица коэффициентов линейной регрессии
4. $w_0$ - постоянный коэффициент
5. $A$ - обратимая матрица, на которую мы умножаем

Тогда модель линейной регресии (1):

$$
y = X \cdot w + w_0
$$

Новая матрица признаков (2):

$$
X' = X \cdot A
$$

И новая модель (3):

$$
y = X' \cdot w' + w_0
$$

Тогда подставим (2) в (3) и получим (4):

$$
y = X \cdot (Aw') + w_0
$$

Здесь $Aw' = w''$ - новый вектор весов. Это можно снова переписать как (5):

$$
y = X \cdot w'' + w_0
$$

Это означает, что после преобразования возможно найти такой новый вектор весов $w''$, при котором вектор предсказаний $y$ остается неизменным.

Обратим внимание, что $w'$ не равен $w$, и только благодаря соотношению $w'' = Aw'$ мы можем делать такие же прогнозы в нашем преобразованном пространстве, как и в исходном пространстве.

Таким образом, веса модели в преобразованном пространстве признаков ($w'$) связаны с весами модели в исходном пространстве признаков ($w$) уравнением $w' = A^{-1}w$ (которое мы получили путем решения уравнения $w'' = Aw'$ относительно $w'$). Стоит отметить, все это возможно только благодаря тому, что $A$ - обратима.

### Реализация преобразования

Напишем функции, которые сделают трансформации за нас:

In [26]:
def get_transform_matrix(features: np.array) -> np.array:
    """ 
    Generates an invertible transformation matrix with the same number of columns as the input features.

    Parameters:
    features (np.array): The feature matrix.

    Returns:
    np.array: An invertible transformation matrix.
    """
    while True:
        transform = np.random.rand(features.shape[1], features.shape[1])
        if np.linalg.det(transform) != 0:
            return transform

In [29]:
get_transform_matrix(df_claims.drop('claims_count', axis=1).values)

array([[0.94172057, 0.76820673, 0.56724618, 0.58249102],
       [0.99372291, 0.81282912, 0.64556645, 0.83300661],
       [0.45011832, 0.82742186, 0.98773322, 0.21645223],
       [0.22891687, 0.24801567, 0.28071993, 0.52171045]])